# Relation extraction with co-occurrences and HuggingFace

In this hands-on session, we'll explore identifying associations between entities in text. We'll first figure out how to find sentences that contain entities of interest, and then explore counting co-occurrences and later more complex relation extraction

**NOTE:** If you are running this with Colab, you should make a copy for yourself. If you don't, you may lose any edits you make. To make a copy, select `File` (top-left) then `Save a Copy in Drive`. If you are not using Colab, you may need to install some prerequisites. Please see the instructions on the [Github Repo](https://github.com/Glasgow-AI4BioMed/ismb2025tutorial).

## Getting Data

As in the previous sessions, we'll download some data that we'll use later on this tutorial with the commands below:

In [ ]:
!wget -O data.zip https://gla-my.sharepoint.com/:u:/g/personal/jake_lever_glasgow_ac_uk/EeRHil_cAVlFo3uarbwXoi0BwlVgIO636LHF0BpIJO6Oug?download=1
!unzip -qo data.zip

## Getting documents with pre-extracted entities

In the last hands-on session, we looked at applying named entity recognition methods to identify mentions of biomedical concepts (e.g. diseases, etc). [PubTator](https://www.ncbi.nlm.nih.gov/research/pubtator3/) is an existing resource which contains entity annotations (for certain types) for PubMed abstracts and PubMed Central full-text articles. Instead of running NER tools in this session, let's look at getting annotated texts from PubTator.

Similar to PubMed, you can get documents through [bulk download](https://ftp.ncbi.nlm.nih.gov/pub/lu/PubTator3) or through [their API](https://www.ncbi.nlm.nih.gov/research/pubtator3/api). Let's examine using their API to get a document. Below is the code to get the text and entity annotations for a single Pubmed abstract (pmid=20573926).

In [ ]:
import requests

# Example PubMed ID
pmid = "20573926"

# PubTator API endpoint for BioC XML
url = f"https://www.ncbi.nlm.nih.gov/research/pubtator-api/publications/export/biocxml?pmids={pmid}"

# Make the request
response = requests.get(url)

# Check for successful response
assert response.status_code == 200

document_xml = response.text

This gets the document in an XML format known as BioC XML. Let's take a quick look at what that looks like. We'll use some code to pretty print the XML which puts in nice indenting for us.

In [ ]:
from xml.dom.minidom import parseString

dom = parseString(document_xml)
print(dom.toprettyxml(indent="  "))

The XML contains documents (with the &lt;document&gt; tag). Each document could contain multiple passages (e.g. the title, the abstract and sections of the paper could be treated as separate passages). Each passage contains text, along with information about the entities annotated there (the &lt;annotation&gt; tag).

Let's save it out to a file for now.

In [ ]:
with open('example.bioc.xml','w') as f:
  f.write(document_xml)

And we'll demonstrate loading a BioC XML file using the [bioc](github.com/bionlplab/bioc) Python package:

In [ ]:
from bioc import biocxml

with open('example.bioc.xml') as f:
  collection = biocxml.load(f)

Let's see how many documents are in this file. We're expecting only one as we downloaded a single one earlier:

In [ ]:
len(collection.documents)

Let's see how many passages are in this single document:

In [ ]:
document = collection.documents[0]
len(document.passages)

Let's see what the text is for each passage:

In [ ]:
for passage in document.passages:
  print(passage.text)
  print("="*30)

Great. It appears to be the title and the abstract as separate passages.

Let's focus on the first passage for now as we explore the data:

In [ ]:
passage = document.passages[0]
passage.text


Now let's see what metadata we can get for this document. With PubTator, we can check the metadata for each passage with `.infons`. It's a dictionary containing various fields including journal, year, etc.

In [ ]:
passage.infons

And how about the annotations? These are the entities that have already been identified by PubTator. These include genes, diseases, chemical, genetic variants and species.

First, how many are there in this short passage?

In [ ]:
len(passage.annotations)

Let's iterate through them and provide some details:

In [ ]:
for anno in passage.annotations:
  print(f"{anno.text=}\n{anno.infons=}\n{anno.total_span.offset=}\n{anno.total_span.length=}\n")

We get similar information to the previous hands-on session with named entity recognition methods. These annotations provide the location of the entities in the text (with the `.total_span.offset` and `.total_span.length` fields). They also provide the entity type with `.infons['Gene']`. And they have done entity linking to various ontologies and resources. The identifier is accessible through `.infons['identifier']`.

## Calculating co-occurrences

We'll start with the most straightforward way to identify relationships between entities - that they appear in lots of documents together. Co-occurrences may not provide nuance (by telling you the specific relation) but they can still be useful to identify that two concepts (e.g. two genes) appear to be connected because they appear together. The co-occurrences could be in appearances together in sentences, paragraphs, paper abstracts or even whole papers. The granularity may depend on what you want to get, and how rare the terms are.

Let's load up a set of documents that have already been annotated through the PubTator platform. We'll use them to calculate some co-occurrence counts. You can load the data with the code below:

In [ ]:
from bioc import biocxml

with open('data/cooccurrences.bioc.xml', "r") as f:
    collection = biocxml.load(f)

The data is in BioC XML data format as we explored above. Let's see how many documents we have:

In [ ]:
len(collection.documents)

That's a lot of documents! You need thousands (or possibly even millions) to get a good signal for identifying co-occurrences, especially for rarer terms.

This dataset comes with text and also the annotations from PubTator. Let's remind ourselves what they look like:

In [ ]:
collection.documents[0].passages[0].annotations[0]

They come with various fields including the text from the research paper, the database identifier that PubTator has linked them to and the entity type. In this case, `cyanogen bromide` has been linked to the MeSH database with ID: `MESH:D003488` and is a `Chemical`. You can use the MeSH Browser to view the page for [MESH:D003488](https://meshb.nlm.nih.gov/record/ui?ui=D003488).

For co-occurrences, it's generally a good idea to use the normalized component (i.e. the ontology/database identifier) instead of the text form (which is 'cyanogen bromide' in this case) as the text form may vary from paper to paper. To make it easier to work with, let's make a lookup to go from the identifier to a text form.

The code below creates a dictionary lookup from the type and identifier to the text:

In [ ]:
quick_lookup = {}
for i,doc in enumerate(collection.documents):
  for passage in doc.passages:

    for anno in passage.annotations:
      quick_lookup[(anno.infons['type'],anno.infons['identifier'])] = anno.text

That means we can lookup the text for the Chemical with identifier `MESH:D003488` with the code below. 

In [ ]:
quick_lookup[('Chemical','MESH:D003488')]

For a full project, it would be better to use the canonical name (i.e. the name that the entity is known by in the database) which would likely involve downloading the database (i.e. getting MeSH) or using an API to lookup the canonical name of entities.

Now let's look at getting some co-occurrences. What entities are in identified in the first document in the collection? The code below outputs it:

In [ ]:
doc = collection.documents[0]

for passage in doc.passages:
  for anno in passage.annotations:
    print(anno.infons['type'], anno.infons['identifier'])

While there are eight annotations, they relate to only two unique entities (a Chemical with identifier `MESH:D003488` and a Gene with identifier `213`). Let's see what those are:

In [ ]:
quick_lookup[('Chemical','MESH:D003488')], quick_lookup[('Gene','213')]

It's the `cyanogen bromide` Chemical that we had before and the `albumin` gene. In this case, we would count this as a single co-occurrence of these two entities, as they have appeared in one document. Alternatively, we could also check whether they appear in the same sentence, but we will stick to the document level at the moment.

Let's look at getting co-occurrence counts at scale:

# Task

Now it's time to go through thousands of documents in this collection and count up the number of documents that entities appear in, and the number of co-occurrences. You should get the identifiers for all the entities mentioned in each document, remove any duplicates (using a Python `set`) and then get every pair of entities. You could use the [itertools.combinations](https://docs.python.org/3/library/itertools.html#itertools.combinations) function to get the pairs.

In [ ]:
import itertools
from collections import Counter

cooccurrences = Counter()

for doc in collection.documents:
  identifiers = [ (anno.infons['type'],anno.infons['identifier']) for passage in doc.passages for anno in passage.annotations ]

  unique_identifiers = sorted(set(identifiers))

  for id1,id2 in itertools.combinations(unique_identifiers, 2):
    cooccurrences[(id1,id2)] += 1


Let's see what the most common pairs are:

In [ ]:
cooccurrences.most_common(5)

Well that's difficult to understand. We can use our `quick_lookup` that we prepared earlier to get text names for each entity from their identifier:

In [ ]:
for (id1,id2),count in cooccurrences.most_common(5):
  print(f"{id1} {quick_lookup[id1]} --- {id2} {quick_lookup[id2]} {count=}")

The most common co-occurrence relates to human patients and cancer, which are both frequently-occurring terms. These counts could be used for a further application to explore the connections with your genes/chemicals/etc of interest.

Often it is used to log-transform co-occurrence values (as some are orders of magnitude larger than others) when dealing with them. But sometimes it's useful to use statistical tests instead.

# Statistical tests for Co-occurrences

There is a good likelihood of two terms appearing together in the same document if one or both are very frequently occurring terms. We could use statistical tests to try to manage this and calculate a P-value for the co-occurrences being statistical chance.

First, we want to know how many documents each term appears in individually. We can refashion the previous code to get those counts for us:

In [ ]:
counts = Counter()

for doc in collection.documents:
  identifiers = [ (anno.infons['type'],anno.infons['identifier']) for passage in doc.passages for anno in passage.annotations ]

  unique_identifiers = sorted(set(identifiers))

  counts += Counter(unique_identifiers)

Let's see what the most common entities are. We're expecting human patients to be up there.

In [ ]:
counts.most_common(5)

Again, those are the entity identifiers from PubTator. Let's use the lookup to see what `9606` is:

In [ ]:
quick_lookup[('Species', '9606')]

As expected, the most frequent term that appears is human patients that appears in 2625 documents.

In [ ]:
doc_count = len(collection.documents)

count_1_and_2 = cooccurrences[(id1, id2)]

count_1_and_not_2 = counts[id1] - count_1_and_2

count_2_and_not_1 = counts[id2] - count_1_and_2

count_not_1_or_2 = doc_count - count_1_and_2 - count_1_and_not_2 - count_2_and_not_1

# 2x2 table
contingency_table = [[count_1_and_2, count_1_and_not_2],
                     [count_2_and_not_1, count_not_1_or_2]]

contingency_table

In [ ]:
count_1_and_2 / (count_1_and_2+count_1_and_not_2)

In [ ]:
counts[id1] / doc_count

In [ ]:
import pandas as pd
from scipy.stats import chi2_contingency

# Chi-squared test
chi2, p, dof, expected = chi2_contingency(contingency_table, correction=False)

p

In [ ]:
for (id1,id2),count in cooccurrences.items():
  pass

## Task

Do that calculation at scale for a big dataset of BioC files

## A rule-based approach

Let's move onto figuring out the relationships between entities. It can be important to know more than if two entities appear in the same context. Is that Chemical causing or treating that Disease?

Sometimes, the text may follow specific patterns (e.g. on drug labels) or we want to extract relations that are worded in highly specific ways. Let's look at using patterns for this.

We'll start by looking at an example sentence:

In [ ]:
sentence = {'text': 'Metformin is used for type 2 diabetes, and studies have evaluated its efficacy in polycystic ovary syndrome.',
 'chemicals': ['Metformin'],
 'diseases': ['type 2 diabetes', 'polycystic ovary syndrome']
}

The sentence follows a common pattern of `[CHEMICAL] is used for [DISEASE]`. Could we programmatically try to match that pattern for this sentence?

In [ ]:
rule = "[CHEMICAL] is used for [DISEASE]"

Let's get all possible pairs of chemical and disease in this sentence:

In [ ]:
pairs = [ (chemical,disease) for chemical in sentence['chemicals'] for disease in sentence['diseases'] ]
pairs

There are two possible pairs of chemical/disease in this sentence. Let's see if any match. We'll focus on the first one (which we do know will match):

In [ ]:
chemical, disease = pairs[0]
chemical, disease

Let's take the sentence and replace the chemical and disease with those placeholders (`[CHEMICAL]` and `[DISEASE]`):

In [ ]:
sentence_with_placeholders = sentence['text'].replace(chemical,'[CHEMICAL]').replace(disease,'[DISEASE]')
sentence_with_placeholders

And then check if there is a match:

In [ ]:
rule_matches = rule in sentence_with_placeholders

print(f"Match: {rule_matches}")
print(f"  [CHEMICAL]={chemical}")
print(f"  [DISEASE]={disease}")
print(f"  {sentence_with_placeholders}")

## Task

The task is to come up with more rules and apply them a dataset of sentences to extract more ways of saying that a chemical treats a disease. The dataset to load is `data/rulebased_sentences.json`. When coming up with rules, you could look at sentences that don't match any rules you've already come up with.

Aim to come up with rules that match with over twenty sentences!


In [ ]:
import json

with open('data/rulebased_sentences.json') as f:
  sentences = json.load(f)

rules = [
  "[CHEMICAL] is used to treat [DISEASE]",
  "[CHEMICAL] treats [DISEASE]",
  "[CHEMICAL] is effective against [DISEASE]",
  "[CHEMICAL] has been shown to treat [DISEASE]",
  "[CHEMICAL] therapy for [DISEASE]",
  "[CHEMICAL] has therapeutic effects on [DISEASE]",
  "[CHEMICAL] is indicated for the treatment of [DISEASE]",
  "[CHEMICAL] is administered to manage [DISEASE]",
  "[CHEMICAL] is prescribed for [DISEASE]",
  "[CHEMICAL] is a treatment option for [DISEASE]",
  "[CHEMICAL] can be used for [DISEASE] therapy",
  "[CHEMICAL] is beneficial for patients with [DISEASE]",
  "Treatment of [DISEASE] with [CHEMICAL]",
  "Use of [CHEMICAL] in the treatment of [DISEASE]",
  "[CHEMICAL] alleviates symptoms of [DISEASE]"
]

counts = Counter()

for sentence in sentences:
  pairs = [ (chemical,disease) for chemical in sentence['chemicals'] for disease in sentence['diseases'] ]

  for chemical,disease in pairs:
    sentence_with_placeholders = sentence['text'].replace(chemical,'[CHEMICAL]').replace(disease,'[DISEASE]')

    if any( rule in sentence_with_placeholders for rule in rules):
      counts[(chemical,disease)] += 1

len(counts)

There are advantages and disadvantages with the rule-based approach:

Advantages:

- The big advantage is that you have full control over how the relations are extracted, which may be very important for your project if specific wording is really needed. 
- It is very very fast. 

Disadvantages:
- Need to write out lots of rules and the guarantee that many wordings will not match any rules.

## A basic Open Information Extraction method

Sometimes you may not be sure what information you want to extract, and certainly couldn't come up with rules to extract things. Open information extraction methods are approaches that attempt to extract information without pre-determined methods. One approach to this is to extract the main verb between two entity mentions. The main verb often describes the action going on and is often the main relation that we care about.

Let's set up an example sentence. We'd really like to extract the `binds` relation between them.

In [ ]:
sentence = {
    "text": "Cetuximab binds to the epidermal growth factor receptor, blocking cancer cell proliferation.",
    "entities": ["Cetuximab", "epidermal growth factor receptor"]
}

Let's figure out where the two entities of interests are in the text (i.e. their string coordinates):

In [ ]:
pair = ("Cetuximab", "epidermal growth factor receptor")

loc1 = sentence['text'].index(pair[0])
loc2 = sentence['text'].index(pair[1])

loc1, loc2

We can use spaCy to parse the text and gets the individual tokens with the parts-of-speech as was shown in the first hands-on session. Let's do that to this sentence:

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")

doc = nlp(sentence['text'])

for token in doc:
  print(f"{token.pos_}: '{token.text}' at {token.idx}")

We want to identify the verb that occurs between our two entities (at positions 0 and 23). Let's adjust the spaCy code to get it

In [ ]:
for token in doc:
    if token.pos_ == "VERB" and token.idx > loc1 and token.idx < loc2:
        print(f"{pair} {token.text}")


You could apply this at scale to find the main verbs between two entities occurring in the same sentence. This has some advantages and disadvantages too:

Advantages:
- You don't need to decide the relation types you want to extract up front
- It can help explore possible information to extract
- It can be fairly fast

Disadvantages:
- It won't catch negative cases (e.g. "Cetuximab never binds to TERT")
- There is more to meaning that the main verb
- The main verb may not be directly connecting the two entities

## Using a machine learning model

A machine learning model can also be used to classify the relationship between two entities in some text. One challenge of this is that you need annotated data to build and evaluate your model.

We will use a model that has been trained on labels generated by a larger language model. This has certain limitations as the labels will likely contain some errors. However, it demonstrates how a machine learning model can be applied. The model we will apply is [Glasgow-AI4BioMed/synthetic_relex](https://huggingface.co/Glasgow-AI4BioMed/synthetic_relex). First, let's load it up:

In [ ]:
from transformers import pipeline

classifier = pipeline("text-classification", model="Glasgow-AI4BioMed/synthetic_relex")


The input to the model is text with the two entities of interest wrapped with [E1][/E1] and `[E2][]

In [ ]:
classifier("[E1]Paclitaxel[/E1] is a common chemotherapy used for [E2]lung cancer[/E2].")

## Task

Apply one of the methods above to the large set of sentences

In [ ]:
with open('data/relex_sentences.json') as f:
  sentences = json.load(f)

In [ ]:
for sentence in sentences:
  entity1, entity2 = sentence['entities']

  doc = nlp(sentence['text'])

  verbs = [ (token.idx, token.text) for token in doc if token.pos_ == "VERB" ]

  loc1 = sentence['text'].index(entity1)
  loc2 = sentence['text'].index(entity2)

  loc1,loc2 = (loc2,loc1) if loc2 < loc1 else (loc1,loc2)

  verbs_between = [ verb for verb_loc,verb in verbs if verb_loc > loc1 and verb_loc < loc2 ]

  if len(verbs_between) == 1:
    print(f"{verbs_between[0]} | {entity1} | {entity2} | {sentence['text']}")



## Optional Extras

- Calculate p-values for each co-occurrence by creating a contigency matrix of document counts of when two entities appear (and appear together)